In [1]:
import os
os.chdir("..")
!pwd
!pyenv version
%load_ext autoreload
%autoreload 2

/Users/heather/code/loicmorel/crypto_assistant
crypto_assistant (set by PYENV_VERSION environment variable)


In [2]:
import pandas as pd
import numpy as np
from app.data_collection.datamgt import ohlcv_from_csv_to_df
from tqdm import trange
from scipy import stats
from sklearn.linear_model import LinearRegression

In [3]:
file = 'data/ohlcv_VITEUSDT_1h_from_1576245600000_to_1651507200000.csv'
data = ohlcv_from_csv_to_df(file)
data

,o_ts,Open,High,Low,Close,Volume,c_ts,o_date
0,2019-12-13 14:00:00,0.02083,0.02400,0.01522,0.01643,29204100.1,2019-12-13 14:59:59.999000064,18243.583333
1,2019-12-13 15:00:00,0.01643,0.01674,0.01556,0.01601,11152260.3,2019-12-13 15:59:59.999000064,18243.625000
2,2019-12-13 16:00:00,0.01602,0.01648,0.01576,0.01589,4090513.3,2019-12-13 16:59:59.999000064,18243.666667
3,2019-12-13 17:00:00,0.01589,0.01648,0.01586,0.01629,1789868.9,2019-12-13 17:59:59.999000064,18243.708333
4,2019-12-13 18:00:00,0.01629,0.01692,0.01609,0.01625,5526217.3,2019-12-13 18:59:59.999000064,18243.750000
...,...,...,...,...,...,...,...,...
20872,2022-05-02 12:00:00,0.04070,0.04096,0.04070,0.04084,185716.7,2022-05-02 12:59:59.999000064,19114.500000
20873,2022-05-02 13:00:00,0.04082,0.04115,0.04018,0.04107,1659280.1,2022-05-02 13:59:59.999000064,19114.541667
20874,2022-05-02 14:00:00,0.04105,0.04133,0.04073,0.04108,355018.9,2022-05-02 14:59:59.999000064,19114.583333
20875,2022-05-02 15:00:00,0.04116,0.04116,0.04071,0.04075,642934.6,2022-05-02 15:59:59.999000064,19114.625000


In [4]:
from app.indicators import find_swings
data_ = find_swings(data)

In [5]:
from app.indicators import classify_swings
data__ = classify_swings(data_)

In [6]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,0.02400,0.01522,0.02400,0.0,0,False,False,0.00000,0.00000,0.00000,0.000,False,False,False,False
1,0.01674,0.01556,0.01674,NaN,0,False,False,0.00000,0.00000,0.00000,0.000,False,False,False,False
2,0.02400,0.00000,0.02400,-1.0,0,False,False,0.00000,0.00000,0.00000,0.000,False,False,False,False
3,0.02400,0.00000,0.02400,-1.0,1,False,True,0.00000,0.02400,0.00000,0.024,False,False,True,False
4,0.02400,0.01576,0.01576,1.0,0,False,False,0.00000,0.00000,0.00000,0.000,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20872,0.04237,0.04083,0.04237,-1.0,4,False,True,0.00000,0.04237,0.00000,0.000,False,False,False,False
20873,0.04237,0.04083,0.04237,-1.0,5,False,True,0.00000,0.04237,0.00000,0.000,False,False,False,False
20874,0.04237,0.04018,0.04018,1.0,0,False,False,0.00000,0.00000,0.00000,0.000,False,False,False,False
20875,0.04237,0.04018,0.04018,1.0,1,True,False,0.04018,0.00000,0.04018,0.000,True,False,False,False


In [7]:
data__=data__*1

In [8]:
data__

,Highs,Lows,Last,Trend,CSLS,Support,Resistance,Strong_lows,Strong_highs,FSL,FSH,LL,HL,HH,LH
0,0.02400,0.01522,0.02400,0.0,0,0,0,0.00000,0.00000,0.00000,0.000,0,0,0,0
1,0.01674,0.01556,0.01674,NaN,0,0,0,0.00000,0.00000,0.00000,0.000,0,0,0,0
2,0.02400,0.00000,0.02400,-1.0,0,0,0,0.00000,0.00000,0.00000,0.000,0,0,0,0
3,0.02400,0.00000,0.02400,-1.0,1,0,1,0.00000,0.02400,0.00000,0.024,0,0,1,0
4,0.02400,0.01576,0.01576,1.0,0,0,0,0.00000,0.00000,0.00000,0.000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20872,0.04237,0.04083,0.04237,-1.0,4,0,1,0.00000,0.04237,0.00000,0.000,0,0,0,0
20873,0.04237,0.04083,0.04237,-1.0,5,0,1,0.00000,0.04237,0.00000,0.000,0,0,0,0
20874,0.04237,0.04018,0.04018,1.0,0,0,0,0.00000,0.00000,0.00000,0.000,0,0,0,0
20875,0.04237,0.04018,0.04018,1.0,1,1,0,0.04018,0.00000,0.04018,0.000,1,0,0,0


In [9]:
df=pd.merge(data,data__,left_index=True,right_index=True)

In [10]:
df=df[['o_ts','Open','High','Low','Close','Volume','LL','HL','HH','LH']]

In [11]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH
0,2019-12-13 14:00:00,0.02083,0.02400,0.01522,0.01643,29204100.1,0,0,0,0
1,2019-12-13 15:00:00,0.01643,0.01674,0.01556,0.01601,11152260.3,0,0,0,0
2,2019-12-13 16:00:00,0.01602,0.01648,0.01576,0.01589,4090513.3,0,0,0,0
3,2019-12-13 17:00:00,0.01589,0.01648,0.01586,0.01629,1789868.9,0,0,1,0
4,2019-12-13 18:00:00,0.01629,0.01692,0.01609,0.01625,5526217.3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
20872,2022-05-02 12:00:00,0.04070,0.04096,0.04070,0.04084,185716.7,0,0,0,0
20873,2022-05-02 13:00:00,0.04082,0.04115,0.04018,0.04107,1659280.1,0,0,0,0
20874,2022-05-02 14:00:00,0.04105,0.04133,0.04073,0.04108,355018.9,0,0,0,0
20875,2022-05-02 15:00:00,0.04116,0.04116,0.04071,0.04075,642934.6,1,0,0,0


In [12]:
df['sum'] = df[['HH','HL','LL','LH']].sum(axis=1)
df['NP']=df['sum'].apply(lambda x:1 if x==0 else 0)
df.drop(columns='sum',inplace=True)

In [13]:
df

,o_ts,Open,High,Low,Close,Volume,LL,HL,HH,LH,NP
0,2019-12-13 14:00:00,0.02083,0.02400,0.01522,0.01643,29204100.1,0,0,0,0,1
1,2019-12-13 15:00:00,0.01643,0.01674,0.01556,0.01601,11152260.3,0,0,0,0,1
2,2019-12-13 16:00:00,0.01602,0.01648,0.01576,0.01589,4090513.3,0,0,0,0,1
3,2019-12-13 17:00:00,0.01589,0.01648,0.01586,0.01629,1789868.9,0,0,1,0,0
4,2019-12-13 18:00:00,0.01629,0.01692,0.01609,0.01625,5526217.3,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
20872,2022-05-02 12:00:00,0.04070,0.04096,0.04070,0.04084,185716.7,0,0,0,0,1
20873,2022-05-02 13:00:00,0.04082,0.04115,0.04018,0.04107,1659280.1,0,0,0,0,1
20874,2022-05-02 14:00:00,0.04105,0.04133,0.04073,0.04108,355018.9,0,0,0,0,1
20875,2022-05-02 15:00:00,0.04116,0.04116,0.04071,0.04075,642934.6,1,0,0,0,0


In [14]:
df.set_index('o_ts',inplace=True)

In [39]:
df.to_csv('data/transformed_data/VITEUSDT_1h_transformed.csv')

In [16]:
df[['HH','HL','LL','LH','NP']].value_counts()

HH  HL  LL  LH  NP
0   0   0   0   1     15948
    1   0   0   0      1450
    0   0   1   0      1409
1   0   0   0   0      1046
0   0   1   0   0      1024
dtype: int64

In [17]:
def subsample_sequence(df, length):
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length`.
    This shorter sequence should be selected at random
    """
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    return df_sample

In [18]:
assert subsample_sequence(df, 10).shape == (10, 10)
assert subsample_sequence(df, 10000).shape == (10000, 10)

In [19]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length -1][['LL','HL','HH','LH','NP']]
    
    X_sample = df_subsample[0:length -1]
    X_sample = X_sample.values
    return np.array(X_sample), np.array(y_sample)

In [20]:
(X_sample, y_sample) = split_subsample_sequence(df, 10)
assert X_sample.shape == (9,10)
assert y_sample.shape == (5,)

In [21]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    return X, y

In [22]:
X, y = get_X_y(df, 1000, 21)
print(X.shape)
print(y.shape)

(1000, 20, 10)
(1000, 5)


In [23]:
assert np.isnan(X).sum() == 0

In [24]:
len_ = int(0.8*df.shape[0])
data_train = df[:len_]
data_test = df[len_:]

In [25]:
X_train, y_train = get_X_y(data_train, 1000, 21)
X_test, y_test = get_X_y(data_test, 200, 21)

In [26]:
assert X_train.shape == (1000, 20, 10)

In [27]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

In [28]:


def init_model():
    opt = optimizers.RMSprop(learning_rate=0.01)
    
    normalizer = Normalization()
    normalizer.adapt(X_train)

    model = models.Sequential()
    model.add(normalizer)
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=True))
    model.add(layers.GRU(units=20, activation='tanh', return_sequences=False))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(5, activation='softmax'))

    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt, 
                  metrics=['accuracy'])
    
    return model

In [29]:
model = init_model()
model.summary()

2022-05-06 12:33:03.775803: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-06 12:33:03.803535: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, None, 10)          21        
_________________________________________________________________
gru (GRU)                    (None, None, 20)          1920      
_________________________________________________________________
gru_1 (GRU)                  (None, 20)                2520      
_________________________________________________________________
dense (Dense)                (None, 50)                1050      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 255       
Total params: 5,766
Trainable params: 5,745
Non-trainable params: 21
_____________________________________________________

In [30]:
from tensorflow.keras.callbacks import EarlyStopping

es=EarlyStopping(monitor='accuracy', patience=10)

In [34]:
class_weight = {0: 5.,
                1: 5.,
                2: 5.,
                3: 5.,
                4: 1.}

In [35]:
model.fit(X_train, y_train, 
          epochs=75, 
          batch_size=64, 
          verbose=1, 
          callbacks = [es],
          validation_split=0.2,class_weight=class_weight)

Epoch 1/75
13/13 [==============================] - 0s 13ms/step - loss: 0.3995 - accuracy: 0.8750 - val_loss: 2.8587 - val_accuracy: 0.6300
Epoch 2/75
13/13 [==============================] - 0s 12ms/step - loss: 0.3625 - accuracy: 0.8988 - val_loss: 2.8826 - val_accuracy: 0.6650
Epoch 3/75
13/13 [==============================] - 0s 11ms/step - loss: 0.3693 - accuracy: 0.9175 - val_loss: 3.0492 - val_accuracy: 0.6350
Epoch 4/75
13/13 [==============================] - 0s 11ms/step - loss: 0.5403 - accuracy: 0.8850 - val_loss: 3.1710 - val_accuracy: 0.5950
Epoch 5/75
13/13 [==============================] - 0s 12ms/step - loss: 0.3200 - accuracy: 0.9200 - val_loss: 2.8726 - val_accuracy: 0.6050
Epoch 6/75
13/13 [==============================] - 0s 12ms/step - loss: 0.6339 - accuracy: 0.8637 - val_loss: 2.9839 - val_accuracy: 0.6300
Epoch 7/75
13/13 [==============================] - 0s 12ms/step - loss: 0.2799 - accuracy: 0.9250 - val_loss: 3.1600 - val_accuracy: 0.6100
Epoch 8/75
13

In [36]:
model.evaluate(X_test, y_test)

7/7 [==============================] - 0s 2ms/step - loss: 4.2080 - accuracy: 0.5900


[4.20802116394043, 0.5899999737739563]

In [37]:
np.round(model.predict_proba(X_test),2)

/Users/heather/.pyenv/versions/3.8.12/envs/crypto_assistant/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


array([[0.96, 0.  , 0.  , 0.  , 0.04],
       [0.09, 0.  , 0.  , 0.  , 0.91],
       [1.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , 0.01, 0.99],
       [0.  , 0.55, 0.  , 0.  , 0.45],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.43, 0.  , 0.  , 0.05, 0.52],
       [0.01, 0.  , 0.  , 0.  , 0.99],
       [0.  , 0.96, 0.  , 0.  , 0.04],
       [0.97, 0.  , 0.  , 0.  , 0.03],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.59, 0.4 ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.11, 0.  , 0.  , 0.89],
       [0.  , 0.  , 0.  , 0.58, 0.42],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  ,

In [38]:
model.save('./models/RNN_VITEUSDT_1h.h5')

2022-05-06 12:34:46.860733: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./models/RNN_VITEUSDT_1h/assets


INFO:tensorflow:Assets written to: ./models/RNN_VITEUSDT_1h/assets
